# Лабораторна робота 1 з "Методів криптоаналізу"
## Тема: Баєсiвський пiдхiд в криптоаналiзi: побудова i дослiдження детермiнiстичної та стохастичної вирiшуючих функцiй.

**Виконали**\
Дигас Богдан, ФІ-52мн\
Юрчук Олексій, ФІ-52мн

In [ ]:
import pandas as pd
import os

In [50]:
def visualize_probability_data(filepath: str):
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found.")
        return

    try:
        df = pd.read_csv(filepath, nrows=2, header=None)
        df.index = ['Opened Texts Probabilities (M)', 'Keys Probabilities (K)']
        df.columns = [f"" for i in range(len(df.columns))]

        print("Probability Data Table:")
        print(df)

    except Exception as e:
        print(f"An error occurred while processing the file: {e}")


In [51]:
file_path = os.path.join('.', 'var_data', 'prob_15.csv')
    
visualize_probability_data(file_path)

Probability Data Table:
                                                                          \
Opened Texts Probabilities (M)  0.11  0.11  0.11  0.11  0.11  0.03  0.03   
Keys Probabilities (K)          0.14  0.14  0.04  0.04  0.04  0.04  0.04   

                                                                          \
Opened Texts Probabilities (M)  0.03  0.03  0.03  0.03  0.03  0.03  0.03   
Keys Probabilities (K)          0.04  0.04  0.04  0.04  0.04  0.04  0.04   

                                                                    
Opened Texts Probabilities (M)  0.03  0.03  0.03  0.03  0.03  0.03  
Keys Probabilities (K)          0.04  0.04  0.04  0.04  0.04  0.04  


In [27]:
def visualize_cipher_table(filepath: str):
    if not os.path.exists(filepath):
        print(f"Error: The file '{filepath}' was not found.")
        return

    try:
        df = pd.read_csv(filepath, header=None)

        row_names = [f"M_{i}" for i in range(df.shape[0])]
        col_names = [f"K_{i}" for i in range(df.shape[1])]
        df.index = row_names
        df.columns = col_names

        print("Cipher table:")
        print(df)

    except Exception as e:
        print(f"An error occurred while processing the file: {e}")

In [28]:
    file_path = os.path.join('.', 'var_data', 'table_15.csv')
    visualize_cipher_table(file_path)

Cipher table:
      K_0  K_1  K_2  K_3  K_4  K_5  K_6  K_7  K_8  K_9  K_10  K_11  K_12  \
M_0     2   14   17   12    3   15    4    5    0    6     9    16     1   
M_1    11   17    4    8   16    3   12   15    0   14     6    19     5   
M_2    13   17   15   11    9   10    2    8   18   14     6    19     3   
M_3    15    9   17   13   19   11   12    2    3   18     1    14     4   
M_4     5    7    4    2   17   13    1   15   12    0     6    14     3   
M_5     8   16    6   11    3   17   14   10   19    7     5    18     4   
M_6     2    6   13   12    1    7    5    0   15   10     8     3     9   
M_7    19   15    4    8   17   18   16    6   10    7     2     1     3   
M_8    13    8   18    4   17    3    1   16   10    5     9    19     2   
M_9     9    6   15   13    0    1   12    2   19   14    18     4     3   
M_10    5   19   12    1    7   17    8    2   16   13     4    14    18   
M_11   11   18   10    9   15    6   13   17    1    7     8     5    19  

In [ ]:
def calculate_probabilities(prob_filepath: str, table_filepath: str) -> pd.Series:

    if not os.path.exists(prob_filepath) or not os.path.exists(table_filepath):
        print("Error: One or both files were not found.")
        return pd.Series(dtype=float)

    try:
        # Row 0: P(M_{i}), Row 1: P(K_{j})
        prob_df = pd.read_csv(prob_filepath, header=None)
        p_plaintexts = prob_df.iloc[0]
        p_keys = prob_df.iloc[1]

        # Rows: Plaintext (M_{i}), Columns: Key (K_{j})
        table_df = pd.read_csv(table_filepath, header=None)

        # Find all unique ciphertext indices
        unique_ciphertexts = table_df.stack().unique()
        unique_ciphertexts.sort()
        
        # Dictionary to store the calculated probabilities for each ciphertext
        p_c_values = {c_idx: 0.0 for c_idx in unique_ciphertexts}

        # Iterate through the table to find all pairs that produce a given ciphertext
        for row_idx, row in table_df.iterrows():
            for col_idx, ciphertext_idx in row.items():
                # Get the probabilities for the current plaintext and key
                p_m_i = p_plaintexts.iloc[row_idx]
                p_k_j = p_keys.iloc[col_idx]
                
                # Add the product to the sum for the corresponding ciphertext
                p_c_values[ciphertext_idx] += p_m_i * p_k_j

        # Convert the dictionary to a pandas Series for cleaner output
        result_series = pd.Series(p_c_values, name="Probabilities P(C)")
        result_series.index = [f"C_{int(idx)}" for idx in result_series.index]
        
        return result_series

    except Exception as e:
        print(f"An error occurred during calculation: {e}")
        return pd.Series(dtype=float)

In [54]:
prob_file = os.path.join('.', 'var_data', 'prob_15.csv')
table_file = os.path.join('.', 'var_data', 'table_15.csv')
probabilities = calculate_probabilities(prob_file, table_file)

if not probabilities.empty:
    print("Розраховані ймовірності P(C) для кожного шифротексту:")
    print(probabilities)
    print("\nСума всіх ймовірностей P(C) має дорівнювати 1:")
    print(f"Sum: {probabilities.sum():.6f}")

Розраховані ймовірності P(C) для кожного шифротексту:
C_0     0.040
C_1     0.043
C_2     0.054
C_3     0.040
C_4     0.043
C_5     0.057
C_6     0.052
C_7     0.051
C_8     0.049
C_9     0.054
C_10    0.040
C_11    0.057
C_12    0.040
C_13    0.057
C_14    0.051
C_15    0.057
C_16    0.049
C_17    0.068
C_18    0.046
C_19    0.052
Name: Probabilities P(C), dtype: float64

Сума всіх ймовірностей P(C) має дорівнювати 1:
Sum: 1.000000


In [63]:
from typing import Tuple

In [64]:
def calculate_ciphertext_probabilities(prob_filepath: str, table_filepath: str) -> Tuple[pd.Series, pd.Series]:
    if not os.path.exists(prob_filepath) or not os.path.exists(table_filepath):
        print("Помилка: Один або обидва файли не знайдено.")
        return pd.Series(dtype=float), pd.Series(dtype=float)

    try:
        prob_df = pd.read_csv(prob_filepath, header=None)
        p_plaintexts = prob_df.iloc[0].rename("P(M)")
        p_keys = prob_df.iloc[1].rename("P(K)")

        table_df = pd.read_csv(table_filepath, header=None)

        unique_ciphertexts = table_df.stack().unique()
        unique_ciphertexts.sort()
        
        # Dictionary to store the calculated probabilities for each ciphertext
        p_c_values = {c_idx: 0.0 for c_idx in unique_ciphertexts}

        # Iterate through the table to find all pairs that produce a given ciphertext
        for row_idx, row in table_df.iterrows():
            for col_idx, ciphertext_idx in row.items():
                p_m_i = p_plaintexts.iloc[row_idx]
                p_k_j = p_keys.iloc[col_idx]
                p_c_values[ciphertext_idx] += p_m_i * p_k_j

        result_series = pd.Series(p_c_values, name="P(C)")
        
        return p_plaintexts, result_series

    except Exception as e:
        print(f"Під час обчислення ймовірностей P(C) сталася помилка: {e}")
        return pd.Series(dtype=float), pd.Series(dtype=float)


def calculate_joint_probabilities(p_plaintexts: pd.Series, p_ciphertexts: pd.Series) -> pd.DataFrame:
    if p_plaintexts.empty or p_ciphertexts.empty:
        print("Помилка: Вхідні дані для обчислення P(M, C) відсутні.")
        return pd.DataFrame()

    joint_prob_df = pd.DataFrame(index=p_plaintexts.index, columns=p_ciphertexts.index)
    
    # Calculate P(M_i, C_j) = P(M_i) * P(C_j)
    for m_idx in p_plaintexts.index:
        for c_idx in p_ciphertexts.index:
            p_m = p_plaintexts.loc[m_idx]
            p_c = p_ciphertexts.loc[c_idx]
            joint_prob_df.loc[m_idx, c_idx] = p_m * p_c

    joint_prob_df.index = [f"M_{i}" for i in joint_prob_df.index]
    joint_prob_df.columns = [f"C_{int(i)}" for i in joint_prob_df.columns]

    return joint_prob_df

In [67]:
prob_file = os.path.join('.', 'var_data', 'prob_15.csv')
table_file = os.path.join('.', 'var_data', 'table_15.csv')

p_plaintexts, p_ciphertexts = calculate_ciphertext_probabilities(prob_file, table_file)
    
if not p_plaintexts.empty and not p_ciphertexts.empty:
    print("Розраховані ймовірності P(C) для кожного шифротексту:")
    print(p_ciphertexts)
    print("\nСума всіх ймовірностей P(C) має дорівнювати 1:")
    print(f"Сума: {p_ciphertexts.sum():.6f}")

    print("\n" + "="*80 + "\n")

    joint_probabilities = calculate_joint_probabilities(p_plaintexts, p_ciphertexts)
        
    if not joint_probabilities.empty:
        print("Таблиця спільних ймовірностей P(M, C):")
        print(joint_probabilities)
        print("\nСума всіх ймовірностей P(М, C) має дорівнювати 1:")
        print(f"Сума: {joint_probabilities.values.sum():.6f}")

Розраховані ймовірності P(C) для кожного шифротексту:
0     0.040
1     0.043
2     0.054
3     0.040
4     0.043
5     0.057
6     0.052
7     0.051
8     0.049
9     0.054
10    0.040
11    0.057
12    0.040
13    0.057
14    0.051
15    0.057
16    0.049
17    0.068
18    0.046
19    0.052
Name: P(C), dtype: float64

Сума всіх ймовірностей P(C) має дорівнювати 1:
Сума: 1.000000


Таблиця спільних ймовірностей P(M, C):
         C_0      C_1      C_2     C_3      C_4      C_5      C_6      C_7  \
M_0   0.0044  0.00473  0.00594  0.0044  0.00473  0.00627  0.00572  0.00561   
M_1   0.0044  0.00473  0.00594  0.0044  0.00473  0.00627  0.00572  0.00561   
M_2   0.0044  0.00473  0.00594  0.0044  0.00473  0.00627  0.00572  0.00561   
M_3   0.0044  0.00473  0.00594  0.0044  0.00473  0.00627  0.00572  0.00561   
M_4   0.0044  0.00473  0.00594  0.0044  0.00473  0.00627  0.00572  0.00561   
M_5   0.0012  0.00129  0.00162  0.0012  0.00129  0.00171  0.00156  0.00153   
M_6   0.0012  0.00129  0.0016

Реалiзувати алгоритми програмно i подати результати побудови детермiнiстичної та стохастичної
вирiшуючих функцiй у виглядi таблиць. Для цього необхiдно:
1. (а) порахувати розподiли P(C) та P(M, C);
2. (б) ґрунтуючись на цих розподiлах обчислити P(M|C);
3. (в) побудова оптимальних детермiнiстичної та стохастичної вирiшуючих функцiй зводиться до
максимiзацiї P(M|C).
4. Обчислити середнi втрати, провести порiвняльний аналiз вирiшуючих функцiй.